# Exercise 03

Written By: Aiden Zelakiewicz (asz39@cornell.edu)

In this exercise, we will go over many useful Python packages for astronomy and some of their features. Three of the big package we will overview are `numpy`, `astropy`, and `scipy`. For `numpy` in particular, I will overview some useful features of the package we did not go over in the prior exercises. We will also splash in some `matplotlib` to visualize many of the concepts we will go over. All of these packages work well together, so you should notice some overlap as we proceed.

## Numpy

In [ ]:
import numpy as np

In [ ]:
# Equally spaced points
x_lin = np.linspace(0, 10, 100)

# Logarithmically spaced points
x_log = np.logspace(0, 1, 100)

# Random points
x_rand = np.random.rand(100)

## Astropy

### Units and Constants

One of the most useful features of `astropy` is its ability to handle units (the module is named `units`). If you assign a variable a unit, you can ask `astropy` to convert it do a different standard or do calculations without having to convert things yourself. Don't tell my professors, but I haven't calculated unit conversions by hand in ages thanks to `astropy`. This module alone made using Python my go-to calculator, especially since they work with `numpy` functions!

Another module, which goes hand-in-hand with `astropy.units` is `astropy.constants`. Astropy has a lot of standard constants set, such as Boltzman's constant or Plank's constant.

There is a slight learning curve with using `astropy.units`, as you need to know how they define all of the units. Most, however, are very intuitive and you can always check the docs if you're confused!

In [3]:
# Usually I load astropy modules in seperately, rather than the entire package
import astropy.units as u
import astropy.constants as c # another common alias for this is 'const'

# Create a unit of length
length = 26.2 * u.m

print(length)

print(length.to(u.cm))

26.2 m
2620.0 cm


## Scipy